In [1]:
from applications import image_denoising
from models import ConvAutoencoder

import os
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [2]:
model = ConvAutoencoder().to(device)

/home/amedvedev/fprojects/python/denoising/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
name_model = "model_1"
name_dataset = "rain800_idcgan"
path_base = "/home/amedvedev/fprojects/data/"

path_save_model = os.path.join("../assets/pt/", f"{name_dataset}_{name_model}.pt")

try:
    model = torch.load(path_save_model).to(device)
except Exception as e:
    print("Error when loading pretrained model. Use custom.")
    model = ConvAutoencoder().to(device)

/home/amedvedev/fprojects/python/denoising/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
BATCH_SIZE = 16

path_save_train_plots = os.path.join("../assets/runs/train_plots", name_dataset, name_model)

path_train = os.path.join(path_base, name_dataset, "training")
path_test = os.path.join(path_base, name_dataset, "test")

path_train_rainy = path_train + "_rainy"
path_test_rainy = path_test + "_rainy"

path_train_normal = path_train + "_normal"
path_test_normal = path_test + "_normal"

(train_noisy_loader, train_normal_loader), (val_noisy_loader, val_normal_loader) = image_denoising.get_loaders(
    path_train_rainy, path_test_rainy, path_train_normal, path_test_normal, BATCH_SIZE, device
)

In [4]:
def plot_step_results():
    image_denoising.save_results(model, device, path_save_train_plots, train_noisy_loader, train_normal_loader, "train",
                                 limit=1, nrow=8, op_count=1)
    image_denoising.save_results(model, device, path_save_train_plots, val_noisy_loader, val_normal_loader, "test",
                                 limit=1, nrow=8, op_count=1)
    print(f"Step results plotted to {path_save_train_plots}.")


In [5]:
EPOCHS = 2
image_denoising.train(model, train_noisy_loader, train_normal_loader, val_noisy_loader, val_normal_loader,
                      EPOCHS, device, path_save=path_save_model, callbacks=[plot_step_results])

Step results plotted to ../assets/runs/train_plots/rain800_idcgan/model_1.


RuntimeError: NVML_SUCCESS == DriverAPI::get()->nvmlInit_v2_() INTERNAL ASSERT FAILED at "../c10/cuda/CUDACachingAllocator.cpp":1123, please report a bug to PyTorch. 

In [6]:
plot_step_results()

In [11]:
image_denoising.save_full_model(model, path_save_model)

In [12]:
path_save_model_onnx = os.path.join("../assets/onnx/", f"{name_dataset}_{name_model}.onnx")
image_denoising.save_onnx_model(model, path_save_model_onnx, device, image_size=(230, 230))

Exported graph: graph(%img : Float(1, 3, 230, 230, strides=[158700, 52900, 230, 1], requires_grad=0, device=cuda:0),
      %all_layers.0.weight : Float(76, 3, 3, 3, strides=[27, 9, 3, 1], requires_grad=1, device=cuda:0),
      %all_layers.0.bias : Float(76, strides=[1], requires_grad=1, device=cuda:0),
      %all_layers.3.weight : Float(38, 76, 2, 2, strides=[304, 4, 2, 1], requires_grad=1, device=cuda:0),
      %all_layers.3.bias : Float(38, strides=[1], requires_grad=1, device=cuda:0),
      %all_layers.6.weight : Float(38, 38, 2, 2, strides=[152, 4, 2, 1], requires_grad=1, device=cuda:0),
      %all_layers.6.bias : Float(38, strides=[1], requires_grad=1, device=cuda:0),
      %all_layers.9.weight : Float(38, 38, 2, 2, strides=[152, 4, 2, 1], requires_grad=1, device=cuda:0),
      %all_layers.9.bias : Float(38, strides=[1], requires_grad=1, device=cuda:0),
      %all_layers.11.weight : Float(38, 76, 2, 2, strides=[304, 4, 2, 1], requires_grad=1, device=cuda:0),
      %all_layers.11.b